In [ ]:
import navis

In [ ]:
#Import the swc files for neuron skeletons
skeletons_s = navis.read_swc (r"D:\Gordon Lab\IR94e figure\sweet vs IR94e morphology comparison\swc\sweet", include_subdirs=True)
skeletons_I = navis.read_swc (r"D:\Gordon Lab\IR94e figure\sweet vs IR94e morphology comparison\swc\IR94e", include_subdirs=True)

In [ ]:
#Creating labels for each skeleton
import numpy as np
labels_I=[f"IR94e{i}" for i in range(1, len(skeletons_I) + 1)]
labels_s=[f"Gr64f{i}" for i in range(1, len(skeletons_s) + 1)]

In [ ]:
#Replace the skeleton IDs with skeleton labels
skeletons_I.set_neuron_attributes(labels_I, 'id', register=True, na='raise')
skeletons_s.set_neuron_attributes(labels_s, 'id', register=True, na='raise')

In [ ]:
#Add both types of neurons to the same list
skeletons = skeletons_I+skeletons_s

In [ ]:
#Convert the skeleton unit into microns
nl_um = skeletons / 1000

#Make dotprops for NBLAST comparison
dps = navis.make_dotprops(nl_um, k=5, resample=False)

#Generate all by all NBLAST comparisons
aba = navis.nblast_allbyall(dps, progress=False)

#Take the mean of the forward and reverse NBLAST scores
aba_mean = (aba + aba.T) / 2

#Calculate the distance from the NBLAST similarity scores
aba_dist = 1 - aba_mean

In [ ]:
#Hierarchical clustering of the neuron skeletons based on the calculated distance
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import linkage, dendrogram, set_link_color_palette

import matplotlib.pyplot as plt
import matplotlib.colors as mcl
import seaborn as sns

set_link_color_palette([mcl.to_hex(c) for c in sns.color_palette('muted', 10)])
aba_vec = squareform(aba_dist, checks=False)
Z = linkage(aba_vec, method='ward')

# Plotting the dendrogram
plt.figure(figsize=(20, 10))
dendrogram_result=dendrogram(Z, labels=aba_mean.columns, leaf_rotation=90, leaf_font_size=15)
plt.tight_layout()
#plt.savefig("Gr64f vs IR94e clustering.pdf", format="pdf", dpi=300)

In [ ]:
#Save the all by all distance matrix in excel
aba_dist.to_excel('Gr64f vs IR94e_abadist.xlsx', index=True)

In [ ]:
#Import the all by all distance matrix file
import pandas as pd
df_aba = pd.read_excel(r"D:\Gordon Lab\IR94e figure\sweet vs IR94e morphology comparison\Gr64f vs IR94e_abadist.xlsx")

In [ ]:
#Reorder the list based on clustering order
df_aba.set_index('query', inplace=True)
ordered_aba = dendrogram_result['leaves'] 
df_aba_ordered = df_aba.iloc[ordered_aba]
df_abatranspose = df_aba_ordered.transpose()
df_aba_final = df_abatranspose.iloc[ordered_aba]
df_aba_final.to_csv('Ordered Gr64f vs IR94e abamean.csv', index=True)

In [ ]:
#Save the ordered list in excel
df_abadist_ordered = pd.read_csv(r"D:\Gordon Lab\IR94e figure\sweet vs IR94e morphology comparison\Ordered Gr64f vs IR94e abadist.csv")

In [ ]:
#Plot the reordered distance matrix heatmap
import seaborn as sns
import matplotlib.pyplot as plt

df_final = df_abadist_ordered.set_index('query')
plt.figure(figsize=(20, 20))
sns.heatmap(df_final, cmap='coolwarm', square = True)
#plt.savefig('Gr64f vs IR94e abadist_heatmap.pdf',dpi=300)